In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set(color_codes=True)

In [3]:
Train_data=pd.read_csv("Train_UWu5bXk.csv")
Test_data=pd.read_csv("Test_u94Q5KV.csv")

In [4]:
Data=Train_data.append(Test_data,sort=False)

In [5]:
Data['Item_Fat_Content'] = Data['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})
Data.groupby('Item_Identifier')['Item_Weight'].mean().head(5)
for i in Data.groupby('Item_Identifier')['Item_Weight'].mean().index:
    Data.loc[Data.loc[:,'Item_Identifier']==i,'Item_Weight']=Data.groupby('Item_Identifier')['Item_Weight'].mean()[i]

In [6]:
Data['Outlet_Type'].value_counts()

Supermarket Type1    9294
Grocery Store        1805
Supermarket Type3    1559
Supermarket Type2    1546
Name: Outlet_Type, dtype: int64

In [7]:
print(Data.Outlet_Size[Data['Outlet_Type']=='Grocery Store'].value_counts())
print(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type1'].value_counts())
print(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type2'].value_counts())

Small    880
Name: Outlet_Size, dtype: int64
Small     3100
High      1553
Medium    1550
Name: Outlet_Size, dtype: int64
Medium    1546
Name: Outlet_Size, dtype: int64


In [8]:
#Data['Outlet_Size'].fillna(Data['Outlet_Size'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Grocery Store'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type1'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type2'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type3'].mode()[0],inplace=True)

In [9]:
for i in Data.groupby('Item_Identifier')['Item_Visibility'].mean().index:
    Data.loc[Data.loc[:,'Item_Identifier']==i,'Item_Visibility']=Data.groupby('Item_Identifier')['Item_Visibility'].mean()[i]

In [10]:
Data['Outlet_Establishment_Year']=2013-Data['Outlet_Establishment_Year']

In [11]:
Data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

In [12]:
Train_data=Data.dropna()

In [13]:
Test_Data=Data[Data['Item_Outlet_Sales'].isnull()]
Test_Data.drop('Item_Outlet_Sales',axis=1,inplace=True)

/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
categorical_list=['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Size','Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year']

In [17]:
le = LabelEncoder()
for i in categorical_list:
    Train_data[i]=le.fit_transform(Train_data[i])
    Train_data[i]=Train_data[i].astype('category')
    Test_Data[i]=le.fit_transform(Test_Data[i])
    Test_Data[i]=Test_Data[i].astype('category')

/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [20]:
print("Train Data")
print(Train_data.head())
print("Test Data")
print(Test_Data.head())

	Train Data
  Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility Item_Type  \
0           FDA15         9.30                0         0.017235         4   
1           DRC01         5.92                1         0.020653        14   
2           FDN15        17.50                0         0.017457        10   
3           FDX07        19.20                1         0.017834         6   
4           NCD19         8.93                0         0.009780         9   

   Item_MRP Outlet_Identifier Outlet_Establishment_Year Outlet_Size  \
0  249.8092                 9                         4           1   
1   48.2692                 3                         0           1   
2  141.6180                 9                         4           1   
3  182.0950                 0                         5           2   
4   53.8614                 1                         7           0   

  Outlet_Location_Type Outlet_Type  Item_Outlet_Sales  
0                    0           1  

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


In [17]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=9, kernel_initializer='Orthogonal', activation='elu'))  
    model.add(Dense(10,activation='elu',kernel_initializer='Orthogonal'))  
    model.add(Dense(1, kernel_initializer='Orthogonal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [18]:
seed = 7
np.random.seed(seed)
estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=12, verbose=0)

In [21]:
estimator.fit(Train_data.drop(['Item_Outlet_Sales','Outlet_Identifier','Item_Identifier'],axis=1),Train_data['Item_Outlet_Sales'], batch_size = 12, epochs = 50)

In [ ]:
train=Train_data.drop(['Item_Outlet_Sales'],axis=1)
predictions=Train_data['Item_Outlet_Sales']
out=[]
for i in range(len(Test_Data)):
    estimator.fit(Train_data.drop(['Item_Outlet_Sales','Outlet_Identifier','Item_Identifier'],axis=1),Train_data['Item_Outlet_Sales'], batch_size = 12, epochs = 50)
    Output=estimator.predict(Test_Data.drop(['Item_Identifier','Outlet_Identifier'],axis=1)[Test_Data.index==i])
    out.append(Output)
    train.append(Test_Data[Test_Data.index==i])
    predictions.append(pd.Series(Output))

In [ ]:
out